In [1]:
import math

import pandas as pd
import mwclient

from lol_fandom import SITE
from lol_fandom import get_leagues, get_tournaments
from lol_fandom import get_scoreboard_games, get_scoreboard_players
from lol_fandom import from_response

pd.set_option('display.max_columns', None)

In [2]:
class Team:
    q = math.log(10) / 400
    def __init__(self, name):
        self.name = name
        self.win = 0
        self.loss = 0
        self.r = 1000
        self.RD = 350

    def get_g(RDi):
        return 1 / math.sqrt(1 + (3 * Team.q ** 2 * RDi ** 2) / math.pi ** 2)

    def get_e(r0, ri, g):
        return 1 / (1 + 10 ** ((g * (r0 - ri)) / -400))

    def get_d(g, e):
        return 1 / (Team.q ** 2 * g ** 2 * e * (1 - e))
    
    def update_point(team1, team2, result):
        # team1 win - result = 1 team1 loss - result = 0
        assert isinstance(team1, Team)
        assert isinstance(team2, Team)

        team1_r = team1.r
        team2_r = team2.r
        team1_RD = team1.RD
        team2_RD = team2.RD

        team1._update_point(team2_r, team2_RD, result)
        team2._update_point(team1_r, team1_RD, 1 - result)

    def _update_point(self, ri, RDi, s):
        if s == 1:
            self.win += 1
        else:
            self.loss += 1

        g_RD = Team.get_g(RDi)
        e = Team.get_e(self.r, ri, g_RD)
        d_2 = Team.get_d(g_RD, e)
        self.r = self.r + Team.q / (1 / self.RD ** 2 + 1 / d_2) * g_RD * (s - e)

        self.RD = math.sqrt((1 / self.RD ** 2 + 1 / d_2) ** -1)

    def get_win_prob(self, opponent):
        return Team.get_e(self.r, opponent.r, Team.get_g(opponent.RD))

    def to_dict(self):
        data = {
            'Win': self.win,
            'Loss': self.loss,
            'WinRate': self.win / (self.win + self.loss) if self.win != 0 else 0,
            'r': self.r,
            'RD': self.RD
        }

        return data


In [3]:
leagues = get_leagues()
leagues

,League,League Short,Region,Level,IsOfficial
0,2015 All-Star Event,2015 ASE,International,Showmatch,Yes
1,2015 International Wildcard Tournament,2015 IWCT,International,Primary,Yes
2,2016 International Wildcard Qualifier,IWCQ,International,Primary,Yes
3,All-Star,All-Star,International,Showmatch,Yes
4,Arena of Legends,AOL,North America,Secondary,No
...,...,...,...,...,...
120,Vietnam Championship Series,VCS,Vietnam,Primary,Yes
121,Volcano League,VL,Latin America,Secondary,Yes
122,World Championship,WCS,International,Primary,Yes
123,World Cyber Arena,WCA,China,Primary,No


In [4]:
target_leagues = ['LCK', 'LPL', 'LEC', 'LCS']

In [5]:
tournaments = pd.DataFrame()
for league in target_leagues:
    t = get_tournaments(f'L.League_Short="{league}" and T.Year=2022')
    tournaments = pd.concat([tournaments, t])
tournaments = tournaments.reset_index(drop=True)
tournaments = tournaments.sort_values(by='DateStart')
tournaments

,Name,OverviewPage,DateStart,Date,League,Region,EventType,StandardName,Split,SplitNumber,TournamentLevel,IsQualifier,IsPlayoffs,IsOfficial,Year,DateStart__precision,Date__precision
4,LPL 2022 Spring,LPL/2022 Season/Spring Season,2022-01-10,2022-03-25,Tencent LoL Pro League,China,Offline/Online,LPL 2022 Spring,Spring,1,Primary,0,0,1,2022,1,1
0,LCK 2022 Spring,LCK/2022 Season/Spring Season,2022-01-12,2022-03-20,LoL Champions Korea,Korea,Offline,LCK 2022 Spring,Spring,1,Primary,0,0,1,2022,1,1
13,LCS 2022 Lock In,LCS/2022 Season/Lock In,2022-01-14,2022-01-30,League of Legends Championship Series,North America,None,LCS 2022 Lock In,Spring,1,Primary,0,1,1,2022,1,1
8,LEC 2022 Spring,LEC/2022 Season/Spring Season,2022-01-14,2022-03-06,LoL European Championship,Europe,Online,LEC 2022 Spring,Spring,1,Primary,0,0,1,2022,1,1
14,LCS 2022 Spring,LCS/2022 Season/Spring Season,2022-02-05,2022-03-27,League of Legends Championship Series,North America,None,LCS 2022 Spring,Spring,1,Primary,0,0,1,2022,1,1
1,LCK 2022 Spring Playoffs,LCK/2022 Season/Spring Playoffs,2022-03-23,2022-04-02,LoL Champions Korea,Korea,Online/Offline,LCK 2022 Spring Playoffs,Spring,1,Primary,0,1,1,2022,1,1
9,LEC 2022 Spring Playoffs,LEC/2022 Season/Spring Playoffs,2022-03-25,2022-04-10,LoL European Championship,Europe,None,LEC 2022 Spring Playoffs,Spring,1,Primary,0,1,1,2022,1,1
5,LPL 2022 Spring Playoffs,LPL/2022 Season/Spring Playoffs,2022-03-26,2022-04-23,Tencent LoL Pro League,China,Online,LPL 2022 Spring Playoffs,Spring,1,Primary,0,1,1,2022,1,1
15,LCS 2022 Spring Playoffs,LCS/2022 Season/Spring Playoffs,2022-04-02,2022-04-24,League of Legends Championship Series,North America,None,LCS 2022 Spring Playoffs,None,None,Primary,0,1,1,2022,1,1
6,LPL 2022 Summer,LPL/2022 Season/Summer Season,2022-06-10,2022-08-14,Tencent LoL Pro League,China,Offline/Online,LPL 2022 Summer,Summer,2,Primary,0,0,1,2022,1,1


In [10]:
teams = {}
for page in tournaments['OverviewPage']:
    scoreboard_games = get_scoreboard_games(f'T.OverviewPage="{page}"')
    scoreboard_games = scoreboard_games.sort_values(by='DateTime UTC', ascending=False).reset_index(drop=True)

    team_names = scoreboard_games[['Team1', 'Team2']].apply(pd.unique)
    team_names = list(set(list(team_names['Team1']) + list(team_names['Team2'])))
    for name in team_names:
        if name not in teams:
            teams[name] = Team(name)

    for row in scoreboard_games.itertuples():
        team1, team2 = row.Team1, row.Team2
        result= 1 if row.WinTeam == team1 else 0
        Team.update_point(teams[team1], teams[team2], result)

ratings = pd.DataFrame(
    data=map(lambda x: x.to_dict(), teams.values()),
    index=teams.keys()
)
ratings = ratings.sort_values(by='r', ascending=False)
ratings

,Win,Loss,WinRate,r,RD
Gen.G,73,23,0.760417,1228.861781,42.562541
G2 Esports,42,20,0.677419,1181.783919,51.708266
T1,76,26,0.745098,1178.892578,45.182113
Rogue (European Team),40,23,0.634921,1141.377280,49.509602
JD Gaming,71,36,0.663551,1132.104004,38.304416
Victory Five,65,30,0.684211,1124.625170,42.084190
Top Esports,78,44,0.639344,1124.533616,34.030679
Fnatic,38,27,0.584615,1110.082072,48.901829
Royal Never Give Up,64,36,0.640000,1109.527285,38.814881
EDward Gaming,63,40,0.611650,1088.479728,37.130618
